### Project Title

### Team Members

### Overview

### Motivation

### Data Sources

### Data Description

### Data Manipulation

### Data Visualization

In [74]:
# install missing packages
%pip install pandas numpy matplotlib seaborn

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Note: you may need to restart the kernel to use updated packages.


In [75]:
# load data
hospital_general_info = pd.read_csv("../dataset/Hospital_General_Information.csv")
promoting_interoperability = pd.read_csv("../dataset/Promoting_Interoperability_Hospital.csv")

In [76]:
print("Hospital General Information Data:")
print(hospital_general_info.head())

Hospital General Information Data:
  Facility ID                    Facility Name                     Address  \
0      010001  SOUTHEAST HEALTH MEDICAL CENTER      1108 ROSS CLARK CIRCLE   
1      010005         MARSHALL MEDICAL CENTERS  2505 U S HIGHWAY 431 NORTH   
2      010006     NORTH ALABAMA MEDICAL CENTER         1701 VETERANS DRIVE   
3      010007         MIZELL MEMORIAL HOSPITAL               702 N MAIN ST   
4      010008      CRENSHAW COMMUNITY HOSPITAL         101 HOSPITAL CIRCLE   

  City/Town State  ZIP Code County/Parish Telephone Number  \
0    DOTHAN    AL     36301       HOUSTON   (334) 793-8701   
1      BOAZ    AL     35957      MARSHALL   (256) 593-8310   
2  FLORENCE    AL     35630    LAUDERDALE   (256) 768-8400   
3       OPP    AL     36467     COVINGTON   (334) 493-3541   
4   LUVERNE    AL     36049      CRENSHAW   (334) 335-3374   

          Hospital Type                           Hospital Ownership  ...  \
0  Acute Care Hospitals  Government - Hospital

In [77]:
sh = hospital_general_info.shape
print(f'hospital_general dataset has {sh[0]} rows and {sh[1]}columns')

hospital_general dataset has 5381 rows and 38columns


In [78]:
# Briefly review 
hospital_general_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5381 entries, 0 to 5380
Data columns (total 38 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Facility ID                                       5381 non-null   object 
 1   Facility Name                                     5381 non-null   object 
 2   Address                                           5381 non-null   object 
 3   City/Town                                         5381 non-null   object 
 4   State                                             5381 non-null   object 
 5   ZIP Code                                          5381 non-null   int64  
 6   County/Parish                                     5381 non-null   object 
 7   Telephone Number                                  5381 non-null   object 
 8   Hospital Type                                     5381 non-null   object 
 9   Hospital Ownership 

In [79]:
# remove the columns related to Footnote
exclude_pattern = 'Footnote '
df_hospital_filtered= hospital_general_info.loc[
    :, ~hospital_general_info.columns.str.contains(exclude_pattern, case=False)
]


In [81]:
#replace abbreviation to state name
state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
    'DC': 'District of Columbia',    
    'PR': 'Puerto Rico',
    'GU': 'Guam',
    'VI': 'U.S. Virgin Islands',
    'AS': 'American Samoa',
    'MP': 'Northern Mariana Islands'
}
df_hospital_filtered ['State']= df_hospital_filtered['State'].map(state_abbrev)

In [82]:
missing_value = df_hospital_filtered['Emergency Services'].isnull().sum()
print(f'The coulumn (Emergency Services) has {missing_value} missing value')
df_hospital_filtered['Emergency Services'].value_counts()

The coulumn (Emergency Services) has 0 missing value


Emergency Services
Yes    4462
No      919
Name: count, dtype: int64

It shows that there have 4462 hospitals with ER and 919 hospital without ER.


In [83]:
# Find the missing value
hospital_col_lst = df_hospital_filtered.columns.to_list()
for _ in hospital_col_lst:
    missing_value = df_hospital_filtered[_].isnull().sum()
    if missing_value:
        print(f'{_} : {missing_value} missing value')

Meets criteria for birthing friendly designation : 3153 missing value
Hospital overall rating footnote : 2860 missing value
MORT Group Footnote : 3654 missing value
Safety Group Footnote : 3358 missing value
READM Group Footnote : 4283 missing value
Pt Exp Group Footnote : 3158 missing value
TE Group Footnote : 4505 missing value


In [84]:
print("Promoting Interoperability Data:")
print(promoting_interoperability.head())

Promoting Interoperability Data:
   Facility ID                    Facility Name                     Address  \
0        10001  SOUTHEAST HEALTH MEDICAL CENTER      1108 ROSS CLARK CIRCLE   
1        10005         MARSHALL MEDICAL CENTERS  2505 U S HIGHWAY 431 NORTH   
2        10006     NORTH ALABAMA MEDICAL CENTER         1701 VETERANS DRIVE   
3        10007         MIZELL MEMORIAL HOSPITAL               702 N MAIN ST   
4        10008      CRENSHAW COMMUNITY HOSPITAL         101 HOSPITAL CIRCLE   

  City/Town State  ZIP Code County/Parish Telephone Number         CEHRT ID  \
0    DOTHAN    AL     36301       HOUSTON   (334) 793-8701  0015CAN28DKT47C   
1      BOAZ    AL     35957      MARSHALL   (256) 593-8310  0015C0HAX4ESQ0D   
2  FLORENCE    AL     35630    LAUDERDALE   (256) 768-8400  0015CW76TRC3SVN   
3       OPP    AL     36467     COVINGTON   (334) 493-3541  0015CFG3Q10HY2V   
4   LUVERNE    AL     36049      CRENSHAW   (334) 335-3374  0015CFG3Q10HY2V   

  Meets criteria 